# Diabetes Classification

## About dataset
- The Behavioral Risk Factor Surveillance System (BRFSS) is a health-related telephone survey that collects data from U.S. residents on their health-related risk behaviors, chronic health conditions, and use of preventive services
- The dataset has been established in 1984 with 15 states, it now collects data from all 50 states, D.C., and 3 U.S. territories
- Over 400,000 adult interviews are completed each year, making it the largest continuous health survey system in the world
- Factors assessed include tobacco use, healthcare coverage, HIV/AIDS knowledge/prevention, physical activity, and fruit/vegetable consumption
- A record in the data corresponds to a single respondent (each from a single household)
- The description of columns can be found in the linked PDF file

#### Features description
| Feature               | Description                                                                  |
|-----------------------|------------------------------------------------------------------------------|
| diabetes              | Subject was told they have diabetes                                          |
| high_blood_pressure   | Subject has high blood pressure                                              |
| high_cholesterol      | Subject has high cholesterol                                                 |
| cholesterol_check     | Subject had cholesterol check within the last five years                     |
| bmi                   | BMI of the subject                                                           |
| smoked_100_cigarettes | Subject has smoked at least 100 cigarettes during their life                 |
| stroke                | Subject experienced stroke during their life                                 |
| coronary_disease      | Subject has/had coronary heart disease or myocardial infarction              |
| exercise              | Subject does regular exercise or physical activity                           |
| consumes_fruit        | Subject consumes fruits at least once a day                                  |
| consumes_vegetables   | Subject consumes vegetables at least once a day                              |
| heavy_alcohol_drinker | Heavy drinkers are defined as adult men having more than 14 drinks per week |
| insurance             | Subject has some kind of health plan (insurance, prepaid plans, ...)         |
| no_doctor_money       | Subject was unable to visit doctor in the past 12 months because of cost     |
| health                | How good is the health of the subject (self rated)                           |
| mental_health         | Number of days in the past month when subject's mental health was not good   |
| physical_health       | Number of days in the past month when subject's physical health was not good |
| climb_difficulty      | Subject has difficulties climbing stairs                                     |
| sex                   | Sex of the subject                                                           |
| age_category          | Age category of the subject                                                  |
| educatation_level     | Highest level of education achieved by the subject                           |
| income                | Income of subject's household                                                |

Load the dataset. All 5 parts are concatenated

In [ ]:
from core import load_dataset

dataset = load_dataset("data")

Do basic preprocessing on columns and categorical values in order to make the dataset more humanly readable.

In [ ]:
from core import process_columns, remove_unusable_diabetes_categories

process_columns(dataset)

# 'Unnamed: 0' is a duplicate column of ID
dataset.drop("Unnamed: 0", axis="columns", inplace=True)

dataset.drop_duplicates(inplace=True)

    # ID is no longer needed after dropping duplicates
dataset.drop("ID", axis="columns", inplace=True)

# Remove rows where target label is missing
dataset = dataset[~dataset["diabetes"].isna()]

# Remove pre diabetes and diabetes in pregnancy categories
dataset_without_extended_diabetes_categories = remove_unusable_diabetes_categories(dataset)

Let's create a baseline model to have something to compare our more advanced models to.

In [ ]:


from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

dataset = dataset_without_extended_diabetes_categories
X, y = dataset.drop("diabetes", axis="columns"), dataset["diabetes"]
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=0)

baseline = DummyClassifier(strategy="uniform", random_state=0)
baseline.fit(train_X, train_y)
print(classification_report(test_y, baseline.predict(test_X), zero_division=0))


Now we create the random forest model

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.calibration import LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

Perceptron()
# Assuming dataset_with_extended_diabetes_categories is a pandas DataFrame
dataset = dataset_without_extended_diabetes_categories

# Splitting dataset into features and target
X = dataset.drop("diabetes", axis=1)
y = dataset["diabetes"]
y = LabelEncoder().fit_transform(y)

# Splitting dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Create preprocessing steps
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OrdinalEncoder())
    ]
)

numerical_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean"))]
)

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, make_column_selector(dtype_include="category")),
        ("num", numerical_transformer, make_column_selector(dtype_exclude="category")),
    ]
)

# Create the pipeline
random_forest_model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            RandomForestClassifier(
                random_state=42,
                class_weight="balanced_subsample",
                max_depth=12,
                max_features="sqrt",
                max_leaf_nodes=50,
                min_samples_leaf=50,
                n_estimators=120,
            ),
        ),
    ]
)
# 
# param_grid = {
#     "classifier__class_weight": ["balanced", "balanced_subsample", None],
#     "classifier__max_depth": [12, 15, 20, None],
#     "classifier__max_features": [None, "sqrt", "log2"],
#     "classifier__min_samples_leaf": [3, 3**2, 3**3, 3**4],
#     "classifier__max_leaf_nodes": [25, 50, 100, None],
#     "classifier__min_samples_leaf": [3, 3**2, 3**3, 3**4],
#     "classifier__n_estimators": [40, 80, 120, 160, 200],
# }
# 
# clf = GridSearchCV(random_forest_model, param_grid, scoring="recall_macro", n_jobs=-1)
# clf.fit(X_train[:3000], y_train[:3000])
# 
# print(clf.best_params_)
# print(classification_report(y_test, clf.predict(X_test)))

random_forest_model.fit(X_train, y_train)

Let's see how our first model performs.

In [ ]:
y_pred = random_forest_model.predict(X_test)

print(classification_report(y_test, y_pred))

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

import seaborn as sns
from sklearn.preprocessing import LabelEncoder


def encode_diabetes_column(diabetes_column):
    """
    Encodes the 'diabetes' column of the dataset.

    Parameters:
    - diabetes_column: pandas.Series, the 'diabetes' column from the dataset.

    Returns:
    - y_encoded: The encoded labels for the 'diabetes' column.
    - class_labels: The original string labels corresponding to the encoded labels.
    """
    encoder = LabelEncoder()
    y_encoded = encoder.fit_transform(diabetes_column)
    class_labels = encoder.classes_  # Stores the original string labels
    return class_labels


class_labels = encode_diabetes_column(dataset["diabetes"])

# Visualize the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=class_labels,
    yticklabels=class_labels,
)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

Now we split the dataset into training and testing.

In [ ]:
from sklearn.model_selection import train_test_split

dataset = dataset_without_extended_diabetes_categories
diabetes_X, diabetes_y = dataset.drop(columns="diabetes"), dataset.diabetes

diabetes_train_X, diabetes_test_X, diabetes_train_y, diabetes_test_y = train_test_split(
    diabetes_X, diabetes_y, test_size=0.2, random_state=42
)

Create a column transformer to handle categorical data for certain models.

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Step 2: Create the ColumnTransformer
# We use 'remainder='passthrough'' to keep the non-categorical columns unchanged
column_transformer = ColumnTransformer(
    transformers=[
        (
            "OHE",
            OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
            make_column_selector(dtype_include="category"),
        )
    ],
    remainder="passthrough",
)

# Step 3: Fit and Transform the Data
# The transformed data will be a NumPy array
transformed_data = column_transformer.fit_transform(dataset)

# Optional: Convert the transformed data back to a DataFrame
# This step requires generating the new column names after transformation
new_columns = column_transformer.get_feature_names_out()

# Creating a new DataFrame with the transformed data and new column names
one_hot_encoded_dataset = pd.DataFrame(transformed_data, columns=new_columns) # type: ignore

Review our one hot encoded dataset.

In [ ]:
one_hot_encoded_dataset.info()

### And next a perceptron.

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(diabetes_X, diabetes_y, test_size=0.2, random_state=42)


perceptron_model = make_pipeline(
    column_transformer,
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    Perceptron()
)

params_grid = {
    "perceptron__class_weight": [None, 'balanced'],
    "perceptron__penalty": [None, 'l2', 'l1', 'elasticnet'],
    "perceptron__alpha": [0.0001, 0.001, 0.01, 0.1],
    "perceptron__eta0": [0.1, 1, 10],
    "perceptron__shuffle": [True, False]
}

clf = GridSearchCV(perceptron_model, params_grid, cv=10, scoring="recall_macro", n_jobs=-1)
#clf.fit(X_train[:2000], y_train[:2000])

#print(clf.best_params_)
#print(classification_report(y_test, clf.predict(X_test), zero_division=0.0))


Based on grid search we choose to create this model.

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, ConfusionMatrixDisplay
from sklearn.metrics import recall_score

perceptron_final_model = make_pipeline(
    column_transformer,
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    Perceptron(alpha=0.0001, class_weight="balanced", eta0=0.1, penalty='l1', shuffle=False)
)

perceptron_final_model.fit(diabetes_train_X[:10000], diabetes_train_y[:10000])

svm_predicted = perceptron_final_model.predict(diabetes_test_X)


print(f"Recall:\t\t{recall_score(diabetes_test_y, svm_predicted, pos_label='Diabetes.YES')}")
print(f"Precision:\t{precision_score(diabetes_test_y, svm_predicted, pos_label='Diabetes.YES')}")
print(f"F1:\t\t{f1_score(diabetes_test_y, svm_predicted, pos_label='Diabetes.YES')}")
print(f"Accuracy:\t{accuracy_score(diabetes_test_y, svm_predicted)}")

# Display confusion matrix
cm = confusion_matrix(diabetes_test_y, svm_predicted)
ConfusionMatrixDisplay(cm, display_labels=perceptron_final_model.classes_).plot()
plt.show()

### Now let's look at the SVM Model

In [ ]:
# # SVM Model

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import KNNImputer


svm_column_transformer = make_column_transformer(
    (MinMaxScaler(), make_column_selector(dtype_include=["int64", "float64"])),
    (OneHotEncoder(sparse_output=False), make_column_selector(dtype_include="category")),
    remainder="passthrough"
)


svm_pipeline = make_pipeline(
    svm_column_transformer,
    KNNImputer(n_neighbors=3),
    MinMaxScaler(feature_range=(0, 1)),
    SVC(kernel="poly", C=8, degree=5, class_weight="balanced")
)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score


params = [
    {
    # "knnimputer__n_neighbors": [1, 2, 3, 5],
    "svc__kernel": ["poly"],
    "svc__C": [0.5, 1, 2, 4, 8, 16],
    "svc__degree": [1, 2, 3, 4, 5, 6],
    },
    {
    # "knnimputer__n_neighbors": [1, 2, 3, 5],
    "svc__kernel": ["rbf"],
    "svc__C": [0.001, 0.05, 0.5, 1, 2, 4, 8, 16, 20],
    "svc__gamma": ["auto", 0.0001, .0002, 0.00025, 0.0003, 0.0005, 0.001, 0.01, 0.1, 1, 2, 4, 8]
    },
    {
    # "knnimputer__n_neighbors": [1, 2, 3, 5],
    "svc__kernel": ["linear"],
    }
]


recall_diabetes = make_scorer(recall_score, pos_label="Diabetes.YES")
grid_search = GridSearchCV(svm_pipeline, cv=3, param_grid=params, scoring=recall_diabetes, verbose=2, n_jobs=3)
# grid_search.fit(diabetes_train_X, diabetes_train_y)
# grid_search.best_score_
# grid_search.best_params # kernel='rbf'; C=0.05; gamma=0.1; n_neighbors=3

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, ConfusionMatrixDisplay


# Final SVM pipeline
svm_pipeline = make_pipeline(
    column_transformer,
    KNNImputer(n_neighbors=3),
    MinMaxScaler(feature_range=(0, 1)),
    SVC(kernel="rbf", C=0.05, gamma=0.1, class_weight="balanced",)
)

svm_pipeline.fit(diabetes_train_X[:10000], diabetes_train_y[:10000])

svm_predicted = svm_pipeline.predict(diabetes_test_X)


print(f"Recall:\t\t{recall_score(diabetes_test_y, svm_predicted, pos_label='Diabetes.YES')}")
print(f"Precision:\t{precision_score(diabetes_test_y, svm_predicted, pos_label='Diabetes.YES')}")
print(f"F1:\t\t{f1_score(diabetes_test_y, svm_predicted, pos_label='Diabetes.YES')}")
print(f"Accuracy:\t{accuracy_score(diabetes_test_y, svm_predicted)}")

# Display confusion matrix
cm = confusion_matrix(diabetes_test_y, svm_predicted)
ConfusionMatrixDisplay(cm, display_labels=svm_pipeline.classes_).plot()
plt.show()